# 03_Processo_clusterizacao (DBSCAN)

## 01_import_bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

## 02_Carga_de_dados

In [2]:
# === Caminho do arquivo parquet
input = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/fontes_tratadas/02_cluster_input.parquet"

In [3]:
df = pd.read_parquet(input)

## 03_Clusterização

In [4]:
df = df.apply(pd.to_numeric, errors="coerce").dropna(axis=1, how="all")

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

In [6]:
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X_scaled)

In [7]:
# === 3. Redução de dimensionalidade com PCA
X_pca = PCA(n_components=30, random_state=42).fit_transform(X_imputed)

In [8]:
# === 4. Clusterização com DBSCAN
dbscan = DBSCAN(eps=3, min_samples=10)
labels = dbscan.fit_predict(X_pca)

In [9]:
# === 5. Anexar os rótulos ao DataFrame original
df["cluster"] = labels

In [10]:
# === 6. Análise básica
print("\n📊 Tamanho de cada cluster:")
print(df["cluster"].value_counts().sort_index())

print("\n📈 Média das variáveis por cluster:")
print(df.groupby("cluster").mean().round(3))


📊 Tamanho de cada cluster:
-1      327
 0    42131
 1       10
 2       14
Name: cluster, dtype: int64

📈 Média das variáveis por cluster:
         area_  area_ comercial  area_ financeira/controladoria  \
cluster                                                           
-1       0.006            0.008                           0.015   
 0       0.848            0.003                           0.003   
 1       0.000            0.000                           0.000   
 2       0.000            0.000                           0.000   

         area_ gestão e alocação de recursos de ti  area_ jurídica  \
cluster                                                              
-1                                           0.015           0.007   
 0                                           0.001           0.000   
 1                                           0.000           0.000   
 2                                           0.000           0.000   

         area_ marketing  area_ novo

In [ ]:
# === 7. Exportar resultados
df.to_parquet("04_resultado_com_clusters_dbscan.parquet", index=False)
print("\n✅ Resultado exportado para: 04_resultado_com_clusters_dbscan.parquet")